# US09

## Setup

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

## Barplot representing monthly water consumption, as a result of the following specifications given by the user: year, time period (StartMonth,EndMonth) and park identification.

For this case, the challenge was turning user input into the range of values that we wanted to display. After we gather the input, we can group the data by year and month range in that order, and show the barplot.

In [ ]:
import calendar
# Carregar os dados do arquivo CSV
dados = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', delimiter=';')

anos_registados = dados["Year"].unique().tolist()
ano_selecionado = int(input("Insert the year for which you want the data:"))
mes_inicio = int(input("Insert the start month for the data:"))
mes_fim = int(input("Insert the end month for the data:"))
if ano_selecionado not in anos_registados:
  raise ValueError("Insert a valid year!")
if mes_inicio < 1 or mes_fim > 12:
  raise ValueError("Month ranges should be between 1 and 12!")
if mes_inicio > mes_fim:
  raise ValueError("End month needs to be larger than start month!")

# Converter a coluna 'Consumption' para float
dados['Consumption'] = dados['Consumption'].str.replace(',', '.').astype(float)

# Agrupar os dados por parque e ano e calcular a média de consumo
dados_agrupados = dados.query("Year == @ano_selecionado and Month >= @mes_inicio and Month <= @mes_fim").groupby(['Park', 'Month']).agg({'Consumption': 'mean'}).reset_index()

# Criar um gráfico de barras para cada parque mostrando a média de consumo de água por mês
plt.figure(figsize=(12, 8))
sns.barplot(x='Month', y='Consumption', hue='Park', data=dados_agrupados)

# Configurar o título e labels
plt.title(f'Monthly water consumption for the year {ano_selecionado} in the months between {calendar.month_abbr[mes_inicio]} and {calendar.month_abbr[mes_fim]}')
plt.ylabel('Water consumption (average)')

# Exibir o gráfico
plt.show()

Here we see that Parque da Cidade shows the highest consumption rates, with the highest consumption values being usually summer months (June to August).

## Average of monthly costs related to water consumption for each park
### As a result of the following specifications given by the user: number of parks to be analyzed, and park identification.

Most of the work was done to guarantee that the user inputs valid values; a number of parks larger than 0 and smaller than the number of unique parks in the file, and that the park names exist in the CSV file. After that, we get the entries for those park names, group them by year and month in that order, and calculate the mean of all the "Consumption" values so we get an average by each month of each year there are records.

In [ ]:
df = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', sep=';', decimal=',')
# User input
parks = df["Park"].unique().tolist()
nParks = int(input("How many parks do you want to analyze:"))
if nParks <= 0 or nParks > len(parks):
  raise ValueError(f"The number of parks to analyze needs to be greater than 0 and smaller than {len(parks)}!")
parkNames = []
for n in range(0,nParks):
  choice = input(f"Input park name:\nThe Park name needs to be one of the following:\n{parks}\n")
  if choice not in parks:
    raise ValueError("Park name not in data file!")
  parkNames.append(choice)
# Analyzis and display
for pn in parkNames:
  meanForPark = df.query("Park == @pn").groupby(["Year", "Month"]).mean("Consumption")["Consumption"]
  print(pn)
  print(meanForPark)

From what we tested, the months with the highest consumption values are usually summer months (June to August) which makes sense.

## The aim is to analyze and compare statistical indicators between the park with the highest and lowest (not null) water consumption.

In [ ]:
df = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', sep=';', decimal=',')
parkConsumption_Max = df.groupby('Park')['Consumption'].sum()
parkHighestConsumption = parkConsumption_Max.idxmax()
parkLowestConsumption = parkConsumption_Max.idxmin()

### Calculate the mean, median, standard deviation, and the coefficient of skewness

After retrieving the names of the parks with the highest and lowest water consumption, we can apply the next formulas directly using the Pandas library built-in functions.

In [ ]:
df = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', sep=';', decimal=',')
dfForParkHighestConsumption = df.query("Park == @parkHighestConsumption")["Consumption"]
print(f"Mean, median, standard deviation and skewness for park with highest consumption: {parkHighestConsumption}")
print(f"Mean: {dfForParkHighestConsumption.mean():.2f}")
print(f"Median: {dfForParkHighestConsumption.median():.2f}")
print(f"Standard Deviation: {dfForParkHighestConsumption.std():.2f}")
print(f"Skewness: {dfForParkHighestConsumption.skew():.2f}")

dfForParkLowestConsumption = df.query("Park == @parkLowestConsumption")["Consumption"]
print(f"Mean, median, standard deviation and skewness for park with lowest consumption: {parkLowestConsumption}")
print(f"Mean: {dfForParkLowestConsumption.mean():.2f}")
print(f"Median: {dfForParkLowestConsumption.median():.2f}")
print(f"Standard Deviation: {dfForParkLowestConsumption.std():.2f}")
print(f"Skewness: {dfForParkLowestConsumption.skew():.2f}")

The large standard deviation for the park with the highest water consumption indicates that the values are very volatile (we see a large spike during the summer months in one of the previous tasks). Both parks show positive skewness indicating that their highest values are higher than the median.

### Build relative and absolute frequency tables (classified data), considering 5 classes;

In [ ]:
data = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', delimiter=';')

data['Consumption'] = data['Consumption'].str.replace(',', '.').astype(float)

grouped_data = data.groupby(['Park', 'Year', 'Month'])

monthly_consumption = grouped_data['Consumption'].sum()

# Calculate daily water Consumption
def calculate_cost(row):
    
    park, year, month = row['Park'], row['Year'], row['Month']
    consumption = row['Consumption']
    
    total_monthly_consumption = monthly_consumption[(park, year, month)]

    total_monthly_consumption = float(total_monthly_consumption)
    
    if total_monthly_consumption <= 1000:
        cost = consumption * 0.7
    else:
        cost = consumption * 0.7 * 1.15
    
    return cost

data['Cost'] = data.apply(calculate_cost, axis=1)

# Define 5 classes based on the cost column 
data['Cost_Class'] = pd.cut(data['Cost'], bins=5)

# Calculte absolute and relative frequency tables
absolut_freq = data['Cost_Class'].value_counts().sort_index()
relative_freq = data['Cost_Class'].value_counts(normalize=True).sort_index()

print("Absolute Frequency Table")
print(absolut_freq)

print("\nRelative Frequency Table")
print(relative_freq)


### For each park, check if the data has outliers
#### Using the outlier definition as values that deviate from the median by more than 1.5 times the interquartile range;

Reusing the variables `parkHighestConsumption` and `parkLowestConsumption`, we had their names to filter through the dataset. After that, it was a matter of calculating the interquartile range with the formula: $$r_q = q_3 − q_1$$ And multiplying by 1.5 to get the upper and lower limits. Then, we just returned the Consumption values that were higher than the upper limit, and lower than the lower limit.

In [ ]:
df = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', sep=';', decimal=',')
dfForParkHighestConsumption = df.query("Park == @parkHighestConsumption")
q1Highest = dfForParkHighestConsumption["Consumption"].quantile(0.25)
q3Highest = dfForParkHighestConsumption["Consumption"].quantile(0.75)
iqrHighest = q3Highest - q1Highest
lowerLimitHighest = q1Highest - (1.5 * iqrHighest)
upperLimitHighest = q3Highest + (1.5 * iqrHighest)
print(dfForParkHighestConsumption[dfForParkHighestConsumption["Consumption"] > upperLimitHighest])

dfForParkLowestConsumption = df.query("Park == @parkLowestConsumption")
q1Lowest = dfForParkLowestConsumption["Consumption"].quantile(0.25)
q3Lowest = dfForParkLowestConsumption["Consumption"].quantile(0.75)
iqrLowest = q3Lowest - q1Lowest
lowerLimitLowest = q1Lowest - (1.5 * iqrLowest)
upperLimitLowest = q3Lowest + (1.5 * iqrLowest)
print(dfForParkLowestConsumption[dfForParkLowestConsumption["Consumption"] > upperLimitLowest])

### Graphically represent data through histograms with 10 and 100 classes

With the names of the parks with the highest and lowest water consumption, we can build histograms with the frequency of the consumption values. We displayed them in a 2x2 grid, with the highest consuming park on the top.

In [ ]:
# Load the data from a CSV file
data = pd.read_csv('us09/Datasets_for_US09_US10_and_US11_20240421/water_consumption.csv', delimiter=';')

# Convert the 'Consumption' column from string to float
data['Consumption'] = data['Consumption'].str.replace(',', '.').astype(float)

dfForParkHighestConsumption = data.query("Park == @parkHighestConsumption")
dfForParkLowestConsumption = data.query("Park == @parkLowestConsumption")
# Create a figure with two subplots (1 row, 2 columns)
fig, ax = plt.subplots(2, 2, figsize=(12, 6))

# Highest consumption park - Histogram with 10 bins
ax[0][0].hist(dfForParkHighestConsumption['Consumption'], bins=10, edgecolor='black')
ax[0][0].set_title(f'{parkHighestConsumption} - Histogram with 10 Classes')
ax[0][0].set_xlabel('Value')
ax[0][0].set_ylabel('Frequency')

# Highest consumption park - Histogram with 100 bins
ax[0][1].hist(dfForParkHighestConsumption['Consumption'], bins=100, edgecolor='black')
ax[0][1].set_title(f'{parkHighestConsumption} - Histogram with 100 Classes')
ax[0][1].set_xlabel('Value')
ax[0][1].set_ylabel('Frequency')

# Lowest consumption park - Histogram with 10 bins
ax[1][0].hist(dfForParkLowestConsumption['Consumption'], bins=10, edgecolor='black')
ax[1][0].set_title(f'{parkLowestConsumption} - Histogram with 10 Classes')
ax[1][0].set_xlabel('Value')
ax[1][0].set_ylabel('Frequency')

# Lowest consumption park - Histogram with 100 bins
ax[1][1].hist(dfForParkLowestConsumption['Consumption'], bins=100, edgecolor='black')
ax[1][1].set_title(f'{parkLowestConsumption} - Histogram with 100 Classes')
ax[1][1].set_xlabel('Value')
ax[1][1].set_ylabel('Frequency')

# Adjust layout and display the plots
plt.tight_layout()
plt.show()


More classes mean that we get more fine-grained data. Each bar represents a smaller range of values so the frequency values are more precise, like with a higher resolution.

# US10

## Make a pie chart representing, in percentage, the use of each piece of equipment.

In order to get the usage for each equipment from a file that just contains one line per use, we first need to aggregate the equipment names and how many times they appear in the file. After we turn it into a DataFrame, we can create a pie chart from it.

In [ ]:
# get equipment names and occurrences
df = pd.read_csv("us10/EquipmentUsed.csv")
equipmentUseCount = df['Equipment'].value_counts().reset_index()
plt.pie(equipmentUseCount['count'], labels=equipmentUseCount['Equipment'], autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')

We conclude that the most used equipments are, in order: walking trails, picnic area, rest benches, and children's playground. We can establish a common patter in the high usage, indicating family activities and leisure rather than exercise.

# US11

## Setup

In [ ]:
df = pd.read_csv('us11/Inquiry.csv', sep=';')

## Indicate the type of each of the three variables

- Escalao: Qualitative/categorical and ordinal
- Y/N: Qualitative/categorical and nominal
- Visits: Quantitative/numeric and discrete

## Indicate the proportion of users from each age group who would recommend the park to others.

To get a proportion of users, we need to get the number of users for each category with a recommendation (value "Y" for the column "Y/N") and divide by the number of users in total.

In [ ]:
df.query("`Y/N` == 'Y'").groupby('Escalao').size() / df.groupby('Escalao').size()

We see that age group 2 (Adult (between 16 and 65 years old)) is more likely to recommend the park to others.

## Create a boxplot for each age group, regarding the monthly frequency of use of the park, and draw the main conclusions obtained from this type of graph.

The CSV file is already formatted in a way that we can read directly without additionally treating the data.

In [ ]:
sns.boxplot(data=df, x='Escalao', y='Visits')

We conclude that age group 3 (Senior (over 65 years old)) visit the park more often on average, and they also have the largest outliers. Which is to say that the people that visited the park the most often during the month, also belonged to the age group 3.

# Contributions for each member of the group

* 1070399: 40%
* 1211742: 40%
* 1040523: 20%